In [130]:
import numpy as np
import pandas as pd

In [131]:
%%time

transactions = pd.read_parquet('data/parquet/transactions_train.parquet')
customers = pd.read_parquet('data/parquet/customers.parquet')
articles = pd.read_parquet('data/parquet/articles.parquet')

CPU times: total: 3.7 s
Wall time: 2.93 s


In [132]:
all_data = transactions.merge(customers, on='customer_id', how='left')
all_data = all_data.merge(articles, on='article_id', how='left')

In [133]:
all_data.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,FN,Active,club_member_status,fashion_news_frequency,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,1728846800780188,519773001,0.028458,2,0,-1,-1,0,0,...,1,0,0,1,0,15,0,1003,3,10231
1,2018-09-20,1728846800780188,578472001,0.032525,2,0,-1,-1,0,0,...,23,0,0,1,0,19,40,1007,9,26053
2,2018-09-20,2076973761519164,661795002,0.167797,2,0,-1,-1,0,0,...,23,0,0,1,0,19,40,1007,9,32892
3,2018-09-20,2076973761519164,684080003,0.101678,2,0,-1,-1,0,0,...,23,0,0,1,0,19,40,1007,9,6151
4,2018-09-20,2918879973994241,662980001,0.033881,1,0,-1,-1,0,0,...,1,0,0,1,0,15,0,1003,3,8172


In [134]:
TEST_WEEK = 104
train = transactions[transactions.week != TEST_WEEK]
test = transactions[transactions.week == TEST_WEEK].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()
unique_customers = pd.DataFrame(test['customer_id'].unique(), columns=['customer_id'])


In [135]:
customers['age_group'] = pd.cut(customers['age'], bins=[0, 25, 40, 60, 100], labels=[0, 1, 2, 3])

avg_price_spent = train.groupby('customer_id')['price'].mean()
max_price_spent = train.groupby('customer_id')['price'].max()

customers['avg_price_spent'] = customers['customer_id'].map(avg_price_spent).fillna(0)
customers['max_price_spent'] = customers['customer_id'].map(max_price_spent).fillna(0)

customers['avg_price_group'] = pd.cut(customers['avg_price_spent'], bins=[-1, 0.02, 0.04, 0.1, 0.3, 1], labels=[0, 1, 2, 3, 4])
customers['max_price_group'] = pd.cut(customers['max_price_spent'], bins=[-1, 0.02, 0.04, 0.1, 0.3, 1], labels=[0, 1, 2, 3, 4])


In [136]:
customers[customers.customer_id.isin(unique_customers.customer_id)].head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_group,avg_price_spent,max_price_spent,avg_price_group,max_price_group
80,8215006181404469139,-1,-1,0,0,27,82312,1,0.029114,0.052525,1,2
86,8282976502227659861,1,1,0,1,33,6721,1,0.028819,0.050831,1,2
107,10842196449521791401,-1,-1,0,0,29,39828,1,0.028978,0.083390,1,2
117,10891223427188082021,1,1,0,1,25,4728,0,0.000000,0.000000,0,0
179,10804975604879079332,-1,-1,0,0,23,130118,0,0.030586,0.067780,1,2


In [137]:
customers[customers.customer_id == 4245900472157]

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_group,avg_price_spent,max_price_spent,avg_price_group,max_price_group
1317296,4245900472157,-1,-1,0,0,21,64922,0,0.021169,0.025407,1,1


In [138]:
train.groupby('customer_id')['price'].mean()

customer_id
4245900472157           0.021169
23962613628581          0.036932
25398598941468          0.042356
28847241659200          0.033151
41046458195168          0.016932
                          ...   
18446630855572834764    0.045181
18446662237889060501    0.054220
18446705133201055310    0.050831
18446723086055369602    0.024347
18446737527580148316    0.020763
Name: price, Length: 1356709, dtype: float32

In [139]:
def candidates_user_feature(feature):
    unique_customers_data = unique_customers.merge(customers, on='customer_id', how='left')
    train_customers = train.merge(customers, on='customer_id', how='left')
    train_customers = train_customers[train_customers['customer_id'].isin(unique_customers['customer_id'])]
    article_counts = train_customers.groupby([feature, 'article_id']).size().reset_index(name='count')
    article_counts_sorted = article_counts.sort_values([feature, 'count'], ascending=[True, False])
    top_articles_feature = article_counts_sorted.groupby(feature).head(12)
    return unique_customers_data.merge(top_articles_feature, on=[feature], how='left')[['customer_id', 'article_id']]

In [140]:
candidates_age_group = candidates_user_feature('age_group')
candidates_avg_price_group = candidates_user_feature('avg_price_group')
candidates_max_price_group = candidates_user_feature('max_price_group')


C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\3337531997.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  article_counts = train_customers.groupby([feature, 'article_id']).size().reset_index(name='count')
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\3337531997.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  top_articles_feature = article_counts_sorted.groupby(feature).head(12)
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\3337531997.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

In [141]:
candidates_age_group.head()

,customer_id,article_id
0,3177658828628418,706016001.0
1,3177658828628418,759871002.0
2,3177658828628418,706016002.0
3,3177658828628418,448509014.0
4,3177658828628418,372860001.0


In [142]:
customers[customers.customer_id == 3177658828628418].age_group

218643    0
Name: age_group, dtype: category
Categories (4, int64): [0 < 1 < 2 < 3]

In [143]:
pivot_table = pd.pivot_table(
    all_data[all_data.week < TEST_WEEK],
    index='customer_id',
    columns='index_code',
    values='article_id',
    aggfunc='count',
    fill_value=0
)

pivot_table['total_purchases'] = pivot_table.sum(axis=1)

# Calculate the percentage of purchases in each category for each customer
pivot_table['percentage_women_purchases'] = ((pivot_table[0] + pivot_table[7] + pivot_table[6]) / pivot_table['total_purchases'])
pivot_table['percentage_children_purchases'] = ((pivot_table[5] + pivot_table[3] + pivot_table[4] + pivot_table[8]) / pivot_table['total_purchases'])
pivot_table['percentage_men_purchases'] = (pivot_table[2] / pivot_table['total_purchases'])

In [144]:
pivot_table.reset_index(inplace=True)
pivot_table = pivot_table[pivot_table['customer_id'].isin(unique_customers['customer_id'])]
pivot_table

index_code,customer_id,0,1,2,3,4,5,6,7,8,9,total_purchases,percentage_women_purchases,percentage_children_purchases,percentage_men_purchases
87,1402273113592184,0,0,0,0,0,0,0,2,0,0,2,1.000000,0.000000,0.000000
124,1827730561464445,19,45,1,0,0,0,2,79,0,0,146,0.684932,0.000000,0.006849
136,1951136007097426,2,5,0,0,0,0,0,1,0,0,8,0.375000,0.000000,0.000000
188,2639747769247776,12,3,10,0,0,1,0,7,0,0,33,0.575758,0.030303,0.303030
224,3177658828628418,0,1,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356590,18444954504588539615,0,0,1,0,0,0,2,10,0,0,13,0.923077,0.000000,0.076923
1356602,18445164350380731040,10,0,0,0,0,0,0,17,0,6,33,0.818182,0.000000,0.000000
1356616,18445340048433064259,18,8,3,0,0,6,3,9,0,3,50,0.600000,0.120000,0.060000
1356631,18445641720816255142,2,2,7,0,0,0,2,6,0,0,19,0.526316,0.000000,0.368421


In [145]:
# Calculate the most bought gender for each customer
pivot_table['most_bought_gender'] = pivot_table[['percentage_women_purchases', 'percentage_children_purchases', 'percentage_men_purchases']].idxmax(axis=1)
customers['most_bought_gender'] = customers['customer_id'].map(pivot_table.set_index('customer_id')['most_bought_gender'])

In [146]:
customers[customers.customer_id.isin(unique_customers.customer_id)]['most_bought_gender'].value_counts()

most_bought_gender
percentage_women_purchases       61201
percentage_men_purchases          1548
percentage_children_purchases      663
Name: count, dtype: int64

In [147]:
candidates_most_bought_gender = candidates_user_feature('most_bought_gender')

In [148]:
candidates_last_purchase = train[(train.week == TEST_WEEK - 1) & (train.customer_id.isin(unique_customers.customer_id))][['customer_id', 'article_id']]


In [149]:
candidates_last_purchase.head()

,customer_id,article_id
31327797,6611639188934298,685814003
31327798,6611639188934298,736049001
31327799,6611639188934298,685813042
31323286,12391957723947919,915529005
31319674,16379701409566518,806131001


In [150]:
bestsellers_all_time = train.groupby('article_id').size().reset_index(name='count').sort_values('count', ascending=False).head(12)
bestsellers_last_week = train[(train.week == TEST_WEEK - 1)].groupby('article_id').size().reset_index(name='count').sort_values('count', ascending=False).head(12)

In [151]:
bestsellers_all_time.head()

,article_id,count
53792,706016001,49958
53793,706016002,34802
1707,372860001,31482
24782,610776002,30003
70054,759871002,26309


In [152]:
bestsellers_last_week.head()

,article_id,count
17033,909370001,1283
12403,865799006,768
17680,918522001,729
17998,924243001,704
456,448509014,609


In [153]:
bestsellers_last_week['bestseller_rank'] = bestsellers_last_week['count'].rank(ascending=False).astype(int)
bestsellers_last_week.head()

,article_id,count,bestseller_rank
17033,909370001,1283,1
12403,865799006,768,2
17680,918522001,729,3
17998,924243001,704,4
456,448509014,609,5


In [154]:
from itertools import product

candidates_bestsellers_all_time = pd.DataFrame(list(product(unique_customers['customer_id'], bestsellers_all_time['article_id'])), columns=['customer_id', 'article_id'])
candidates_bestsellers_last_week = pd.DataFrame(list(product(unique_customers['customer_id'], bestsellers_last_week['article_id'])), columns=['customer_id', 'article_id'])

In [155]:
merged_candidates = pd.concat([candidates_age_group, candidates_avg_price_group, candidates_max_price_group, candidates_most_bought_gender, candidates_last_purchase, candidates_bestsellers_all_time, candidates_bestsellers_last_week]).drop_duplicates()

In [156]:
methods = ["bestsellers all time", "bestsellers last week (radek)", "previous purchases (radek)", "popular by age group", "popular by avg price", "popular by max price", "popular by most bought gender", "all candidates combined"]
candidates = [candidates_bestsellers_all_time, candidates_bestsellers_last_week, candidates_last_purchase, candidates_age_group, candidates_avg_price_group, candidates_max_price_group, candidates_most_bought_gender, merged_candidates]

for method, cands in zip(methods, candidates):
    merged = test.merge(cands, on=['customer_id', 'article_id'], how='inner')
    counts = merged.groupby('customer_id')['article_id'].nunique()
    totals = test.groupby('customer_id')['article_id'].nunique()
    percentages = counts / totals * 100
    percentages.fillna(0, inplace=True)
    avg_percentage = percentages.mean()
    
    customer_percentages = percentages.reset_index().merge(customers, on='customer_id', how='left')
    percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
    percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
    percentage_by_max_price_group = customer_percentages.groupby('max_price_group')['article_id'].mean()
    percentage_by_most_bought_gender = customer_percentages.groupby('most_bought_gender')['article_id'].mean()
    print(f"{method.upper()}:\nAverage % of candidates bought overall: {avg_percentage:.2f}%\nAverage % by age group:\n{percentage_by_age_group}\nAverage % by avg price group:\n{percentage_by_avg_price_group}\nAverage % by max price group:\n{percentage_by_max_price_group}\nAverage % by age group:\n{percentage_by_age_group}\nAverage % by most bought gender:\n{percentage_by_most_bought_gender}\n\n\n")
    


C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass obse

BESTSELLERS ALL TIME:
Average % of candidates bought overall: 0.77%
Average % by age group:
age_group
0    1.100304
1    0.688329
2    0.601385
3    0.273876
Name: article_id, dtype: float64
Average % by avg price group:
avg_price_group
0    0.865175
1    0.788028
2    0.416613
3    0.000000
4    0.000000
Name: article_id, dtype: float64
Average % by max price group:
max_price_group
0    0.800631
1    1.149788
2    0.822542
3    0.485207
4    0.268519
Name: article_id, dtype: float64
Average % by age group:
age_group
0    1.100304
1    0.688329
2    0.601385
3    0.273876
Name: article_id, dtype: float64
Average % by most bought gender:
most_bought_gender
percentage_children_purchases    0.930944
percentage_men_purchases         0.591534
percentage_women_purchases       0.780661
Name: article_id, dtype: float64





C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass obse

BESTSELLERS LAST WEEK (RADEK):
Average % of candidates bought overall: 2.55%
Average % by age group:
age_group
0    3.126027
1    2.332262
2    2.157088
3    2.703914
Name: article_id, dtype: float64
Average % by avg price group:
avg_price_group
0    2.084097
1    2.573451
2    3.560900
3    6.166285
4    0.000000
Name: article_id, dtype: float64
Average % by max price group:
max_price_group
0    2.475026
1    2.221011
2    2.479007
3    2.905394
4    2.969608
Name: article_id, dtype: float64
Average % by age group:
age_group
0    3.126027
1    2.332262
2    2.157088
3    2.703914
Name: article_id, dtype: float64
Average % by most bought gender:
most_bought_gender
percentage_children_purchases    1.905890
percentage_men_purchases         2.343076
percentage_women_purchases       2.565901
Name: article_id, dtype: float64





C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass obse

...

C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass obse

POPULAR BY MAX PRICE:
Average % of candidates bought overall: 0.92%
Average % by age group:
age_group
0    1.227372
1    0.799701
2    0.799517
3    0.519217
Name: article_id, dtype: float64
Average % by avg price group:
avg_price_group
0    0.808107
1    0.980137
2    0.714015
3    0.588235
4    0.000000
Name: article_id, dtype: float64
Average % by max price group:
max_price_group
0    0.448296
1    1.176352
2    1.025290
3    0.732952
4    1.070370
Name: article_id, dtype: float64
Average % by age group:
age_group
0    1.227372
1    0.799701
2    0.799517
3    0.519217
Name: article_id, dtype: float64
Average % by most bought gender:
most_bought_gender
percentage_children_purchases    0.819079
percentage_men_purchases         0.691336
percentage_women_purchases       0.980693
Name: article_id, dtype: float64





C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass obse

POPULAR BY MOST BOUGHT GENDER:
Average % of candidates bought overall: 1.03%
Average % by age group:
age_group
0    1.359726
1    0.822880
2    0.963426
3    0.689317
Name: article_id, dtype: float64
Average % by avg price group:
avg_price_group
0    0.726016
1    1.168652
2    0.609150
3    0.000000
4    0.000000
Name: article_id, dtype: float64
Average % by max price group:
max_price_group
0    0.181131
1    1.581514
2    1.183604
3    0.757711
4    0.305556
Name: article_id, dtype: float64
Average % by age group:
age_group
0    1.359726
1    0.822880
2    0.963426
3    0.689317
Name: article_id, dtype: float64
Average % by most bought gender:
most_bought_gender
percentage_children_purchases    0.560873
percentage_men_purchases         1.579279
percentage_women_purchases       1.116825
Name: article_id, dtype: float64



ALL CANDIDATES COMBINED:
Average % of candidates bought overall: 5.90%
Average % by age group:
age_group
0    6.168768
1    5.755759
2    5.651452
3    6.638033
Name

C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_age_group = customer_percentages.groupby('age_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentage_by_avg_price_group = customer_percentages.groupby('avg_price_group')['article_id'].mean()
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2486937849.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass obse

### Combining new candidates with radeks ranker

In [157]:
merged_candidates = merged_candidates.dropna(subset=['article_id'])
merged_candidates['article_id'] = merged_candidates['article_id'].astype(int)


C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\1315852610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['article_id'] = merged_candidates['article_id'].astype(int)


In [158]:
merged_candidates['t_dat'] = test['t_dat'].unique()[0]
merged_candidates['week'] = TEST_WEEK


C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2635938015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['t_dat'] = test['t_dat'].unique()[0]
C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\2635938015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['week'] = TEST_WEEK


In [159]:
train_unique_articles = train.groupby('article_id')[['price', 'sales_channel_id']].agg(lambda x: x.mode()[0]).reset_index()

In [160]:
merged_candidates = merged_candidates.merge(train_unique_articles, on='article_id', how='left')

In [161]:
merged_candidates.head()

,customer_id,article_id,t_dat,week,price,sales_channel_id
0,3177658828628418,706016001,2020-09-16,104,0.033881,2
1,3177658828628418,759871002,2020-09-16,104,0.005068,2
2,3177658828628418,706016002,2020-09-16,104,0.033881,2
3,3177658828628418,448509014,2020-09-16,104,0.042356,2
4,3177658828628418,372860001,2020-09-16,104,0.013542,1


In [162]:
transactions['purchased'] = 1

In [163]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
25784,2018-09-20,1728846800780188,519773001,0.028458,2,0,1
25785,2018-09-20,1728846800780188,578472001,0.032525,2,0,1
5389,2018-09-20,2076973761519164,661795002,0.167797,2,0,1
5390,2018-09-20,2076973761519164,684080003,0.101678,2,0,1
47429,2018-09-20,2918879973994241,662980001,0.033881,1,0,1


In [164]:
data = pd.concat([transactions, merged_candidates])
data.purchased.fillna(0, inplace=True)

In [165]:
data.purchased.mean()

0.9366790992154876

In [174]:
data = pd.merge(
    data,
    bestsellers_last_week[['article_id', 'bestseller_rank']],
    on=['article_id'],
    how='left'
)

In [167]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [168]:
data.purchased.mean()

0.9296191207047906

In [175]:
data.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank
0,2018-10-03,28847241659200,691177002,0.042356,1,2,1.0,NaN
1,2018-10-03,481212607390045,465655010,0.016932,2,2,1.0,NaN
2,2018-10-03,2918879973994241,650696002,0.042356,1,2,1.0,NaN
3,2018-10-03,2918879973994241,665481001,0.050831,1,2,1.0,NaN
4,2018-10-03,6273203381383334,446224011,0.014390,2,2,1.0,NaN


In [176]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [177]:
agg_price_spent = transactions.groupby('customer_id')['price'].agg(['mean', 'max']).reset_index()
agg_price_spent.rename(columns={'mean': 'avg_price_spent', 'max': 'max_price_spent'}, inplace=True)

In [178]:
agg_price_spent

,customer_id,avg_price_spent,max_price_spent
0,4245900472157,0.021169,0.025407
1,23962613628581,0.036932,0.067780
2,25398598941468,0.042356,0.042356
3,28847241659200,0.033151,0.128797
4,41046458195168,0.016932,0.016932
...,...,...,...
1362276,18446630855572834764,0.045181,0.067780
1362277,18446662237889060501,0.054220,0.084729
1362278,18446705133201055310,0.050831,0.050831
1362279,18446723086055369602,0.024347,0.031763


In [179]:
transactions_with_age = pd.merge(transactions, customers[['customer_id', 'age']], on='customer_id', how='left')
avg_purchaser_age = transactions_with_age.groupby('article_id')['age'].mean().reset_index()
avg_purchaser_age.rename(columns={'age': 'avg_purchaser_age'}, inplace=True)

In [180]:
avg_purchaser_age

,article_id,avg_purchaser_age
0,108775015,34.193801
1,108775044,35.884828
2,108775051,35.395349
3,110065001,37.939655
4,110065002,39.185529
...,...,...
104542,952267001,34.243902
104543,952938001,36.111111
104544,953450001,31.764706
104545,953763001,35.457143


In [181]:
customers = pd.merge(customers, agg_price_spent, on='customer_id', how='left')
articles = pd.merge(articles, avg_purchaser_age, on='article_id', how='left')

In [182]:
articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,avg_purchaser_age
0,108775015,108775,12855,253,9,0,1010016,0,9,0,...,0,0,1,0,16,30,1002,2,8834,34.193801
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,0,0,1,0,16,30,1002,2,8834,35.884828
2,108775051,108775,44846,253,9,0,1010017,3,11,11,...,0,0,1,0,16,30,1002,2,8834,35.395349
3,110065001,110065,8159,306,13,4,1010016,0,9,0,...,7,7,1,0,61,5,1017,4,8243,37.939655
4,110065002,110065,8159,306,13,4,1010016,0,10,2,...,7,7,1,0,61,5,1017,4,8243,39.185529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,24043,302,14,7,1010014,6,9,0,...,2,2,3,3,26,16,1021,13,28026,31.764706
105538,953763001,953763,24027,253,9,0,1010016,0,9,0,...,0,0,1,0,2,15,1005,0,28025,35.457143
105539,956217002,956217,24040,265,1,2,1010016,0,9,0,...,0,0,1,0,18,12,1005,0,28024,48.428571
105540,957375001,957375,24041,72,48,3,1010016,0,9,0,...,1,1,2,2,52,25,1019,1,28023,NaN


In [183]:
articles.avg_purchaser_age.fillna(-1, inplace=True)

In [184]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [185]:
articles['index_code'].drop_duplicates()

0       0
3       7
11      5
16      2
28      6
40      9
69      3
78      1
81      4
1367    8
Name: index_code, dtype: int8

In [186]:
pivot_table = pd.pivot_table(
    data,
    index='customer_id',
    columns='index_code',
    values='article_id',
    aggfunc='count',
    fill_value=0
)

pivot_table['total_purchases'] = pivot_table.sum(axis=1)

# Calculate the percentage of purchases in each category for each customer
pivot_table['percentage_women_purchases'] = ((pivot_table[0] + pivot_table[7] + pivot_table[6]) / pivot_table['total_purchases'])
pivot_table['percentage_children_purchases'] = ((pivot_table[5] + pivot_table[3] + pivot_table[4] + pivot_table[8]) / pivot_table['total_purchases'])
pivot_table['percentage_men_purchases'] = (pivot_table[2] / pivot_table['total_purchases'])

In [187]:
pivot_table.reset_index(inplace=True)
pivot_table

index_code,customer_id,0,1,2,3,4,5,6,7,8,9,total_purchases,percentage_women_purchases,percentage_children_purchases,percentage_men_purchases
0,4245900472157,1,1,0,0,0,0,0,0,0,0,2,0.500000,0.0,0.000000
1,23962613628581,10,9,0,0,0,0,0,0,0,0,19,0.526316,0.0,0.000000
2,25398598941468,0,1,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000
3,28847241659200,31,11,0,0,0,0,4,13,0,7,66,0.727273,0.0,0.000000
4,41046458195168,0,0,0,0,0,0,0,2,0,0,2,1.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336806,18446630855572834764,4,0,0,0,0,0,0,0,0,0,4,1.000000,0.0,0.000000
1336807,18446662237889060501,2,2,0,0,0,0,0,0,0,0,4,0.500000,0.0,0.000000
1336808,18446705133201055310,1,0,0,0,0,0,0,0,0,0,1,1.000000,0.0,0.000000
1336809,18446723086055369602,0,0,0,0,4,0,0,0,0,0,4,0.000000,1.0,0.000000


In [188]:
data = pd.merge(data, pivot_table[['customer_id', 'total_purchases', 'percentage_women_purchases', 'percentage_children_purchases', 'percentage_men_purchases']], on='customer_id', how='left')

In [189]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [191]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [192]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank', 
'age_group', 'avg_purchaser_age', 'percentage_children_purchases', 'percentage_men_purchases', 'percentage_women_purchases', 'total_purchases']

In [193]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: total: 2.27 s
Wall time: 2.29 s


In [194]:
from lightgbm.sklearn import LGBMRanker

In [195]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [196]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.889407
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.185605
[LightGBM] [Debug] init for col-wise cost 0.586123 seconds, init for row-wise cost 2.195363 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.722216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2401
[LightGBM] [Info] Number of data points in the train set: 27203211, number of used features: 24
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 1 and depth = 1
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
CPU times: total: 1min 12s
Wall time: 20.2 s


In [197]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

total_purchases nan
percentage_women_purchases nan
product_type_no nan
graphical_appearance_no nan
colour_group_code nan
perceived_colour_value_id nan
perceived_colour_master_id nan
department_no nan
index_code nan
index_group_no nan
section_no nan
garment_group_no nan
FN nan
Active nan
club_member_status nan
fashion_news_frequency nan
age nan
postal_code nan
bestseller_rank nan
age_group nan
avg_purchaser_age nan
percentage_children_purchases nan
percentage_men_purchases nan
article_id nan


C:\Users\5joshi\AppData\Local\Temp\ipykernel_19832\1815302757.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())


In [198]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()


CPU times: total: 0 ns
Wall time: 0 ns


In [199]:
sub = pd.read_csv('data/original/sample_submission.csv')

In [200]:
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

In [208]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

ValueError: Unable to coerce to Series, length must be 3: given 0

In [ ]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [ ]:
sub_name = 'candidate_generation_model'
sub.to_csv(f'{sub_name}.csv.gz', index=False)